In [0]:

from pyspark.sql import functions as F







creative_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-market/unified/market.creative-fact.v2/fact/granularity=monthly/date=2020-01-31/").groupBy("ad_app_id", "country_code", "ad_platform_id", "device_code").agg(F.collect_set("creative_id").alias('creative_id_list'))

creative_df = creative_df.withColumn("creative_count", F.size("creative_id_list"))


print "creative_cout",creative_df.count()

ad_app_creative = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-market/unified/market.adapp-fact.v2/fact/granularity=monthly/date=2020-01-31/").select("ad_app_id", "country_code", "ad_platform_id", "device_code", "creative_count")

print "add_app_count",ad_app_creative.count()



merge_df = creative_df.alias("creative").join(ad_app_creative.alias("adapp"), [creative_df.device_code==ad_app_creative.device_code, creative_df.ad_platform_id==ad_app_creative.ad_platform_id, creative_df.country_code==ad_app_creative.country_code, creative_df.ad_app_id==ad_app_creative.ad_app_id,creative_df.creative_count != ad_app_creative.creative_count], how='outer')



print "merged_df count",merge_df.drop_duplicates().count()






In [0]:

merged_cnt = merge_df.filter("adapp.ad_app_id is not null and creative.ad_app_id is not null").count()
print merged_cnt

In [0]:
%%sh
PGPASSWORD='rn*Wh%osCl2C' psql -h internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com  -U citus_mkt_qa -d aa_mkt_db -p 7432 << EOF

select creative_count from market.market_adapp_fact_v1 where 
            granularity='monthly' AND device_code = 'ios-phone' AND  
            country_code = 'BR' AND date='2020-03-31' AND ad_app_id=284035177 and ad_platform_id=325;

select count(distinct(creative_id)) from  market.market_creative_adapp_fact_v1 where 
            granularity='monthly' AND  device_code = 'ios-phone' AND country_code ='BR'
            AND date='2020-03-31' AND ad_app_id=284035177 and ad_platform_id=325;
EOF


In [0]:

from pyspark.sql import functions as F
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf
from collections import defaultdict
import re





CREATIVE_BASE_PATH = "s3://b2c-prod-data-pipeline-unified-market/unified/market.creative-fact.v2/fact/granularity={}/date={}/"
ADAPP_BASE_PATH = "s3://b2c-prod-data-pipeline-unified-market/unified/market.adapp-fact.v2/fact/granularity={}/date={}/"
GRANULARITY = ["weekly", "monthly"]
def _parse_string(path):
    pattern = re.compile("unified.*?granularity=(.*?)/date=(.*?)/")
    return pattern.search(path).groups()[0], pattern.search(path).groups()[1]

date_range = defaultdict(list)
s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-market'))
for _granu in GRANULARITY:
    path_list = s3_bucket_list.all(prefix="unified/market.adapp-fact.v2/fact/granularity={granularity}".format(granularity=_granu), depth_is_1=True)
    for _path in path_list:
        date_range[_parse_string(_path)[0]].append(_parse_string(_path)[1])



def test_creative_count(granularity, date):
    creative_df = spark.read.format("delta").option("timestampAsOf", "2020-04-10").load(CREATIVE_BASE_PATH.format(granularity, date)).filter("ad_platform_id != 1 and country_code != 'WW'").groupBy("ad_app_id", "country_code", "ad_platform_id", "device_code").agg(F.collect_set("creative_id").alias('creative_id_list'))

    creative_df = creative_df.withColumn("creative_count", F.size("creative_id_list"))

    # print "creative_cout",creative_df.count()

    adapp_df = spark.read.format("delta").option("timestampAsOf", "2020-04-10").load(ADAPP_BASE_PATH.format(granularity, date)).filter("ad_platform_id != 1 and country_code != 'WW'").select("ad_app_id", "country_code", "ad_platform_id", "device_code", "creative_count")

    # print "add_app_count",ad_app_creative.count()

    merged_df = creative_df.alias("creative").join(adapp_df.alias("adapp"), [creative_df.device_code==adapp_df.device_code, creative_df.ad_platform_id==adapp_df.ad_platform_id, creative_df.country_code==adapp_df.country_code, creative_df.ad_app_id==adapp_df.ad_app_id,creative_df.creative_count != adapp_df.creative_count], how='outer')

    # print "merged_df count",merge_df.drop_duplicates().count()
    # merged_df.show()
    merged_cnt = merged_df.filter("adapp.ad_app_id is not null and creative.ad_app_id is not null").count()
    
    
    # merged_df.filter("adapp.ad_app_id is not null and creative.ad_app_id is not null").show()
    # if granularity=='monthly':
    #     try:
    #         assert (merged_df.filter("adapp.ad_app_id is not null and creative.ad_app_id is not null and creative.creative_count > adapp.creative_count").count()) == 0,"{} - {} - count !=0".format(granularity, date)
    #     except AssertionError as msg:
    #         print msg
    
    # elif granularity=='weekly':
    #     try:
    #         assert (merged_df.filter("adapp.ad_app_id is not null and creative.ad_app_id is not null and creative.creative_count < adapp.creative_count").count()) == 0,"{} - {} - count !=0".format(granularity, date)
    #     except AssertionError as msg:
    #         print msg
    # print merged_cnt
    try:
        assert merged_cnt == 0,"In {} - {} - mismatched data is: {}".format(date, granularity, merged_cnt)
    except AssertionError as msg:
        print msg

def main():
    for granularity, dates in date_range.items():
        for date in dates:
            # print granularity, date, "is under checking"
            test_creative_count(granularity, date)
    print "done"
# test_creative_count("monthly", "2016-08-31")
main()

In [0]:

r = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-market/unified/market.creative-fact.v2/fact/granularity=monthly/").filter("ad_platform_id != 1 and country_code != 'WW'").select("ad_app_id").distinct().count()
print "creative:", r
spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-market/unified/market.adapp-fact.v2/fact/granularity=monthly/date=2019-03-31/").filter("country_code='NO' and device_code='ios-tablet' and ad_app_id=1217381921 and ad_platform_id=1").select("country_code","device_code", "ad_app_id", "ad_platform_id", "creative_count").show()

In [0]:

BASE_BACKET = 'b2c-prod-data-pipeline-unified-market'
CREATIVE_PREFIX = "unified/market.creative-fact.v2/fact/granularity={}"
BASE_CNT_SQL = """SELECT COUNT(DISTINCT {dimension}) from  market.market_creative_adapp_fact_v2 where granularity='{granularity}' and date='{date}';"""
AD_APP_ID = 'ad_app_id'
COUNTRY_CODE = 'country_code'
AD_PLATFORM_ID = 'ad_platform_id'


def load_base_test():
    for _g in GRANULARITY:
        s3_bucket_list = s3.S3Bucket(Conf(bucket_name=BASE_BACKET))
        path_list = s3_bucket_list.all(prefix=CREATIVE_PREFIX.format(_g), depth_is_1=True)
        for _p in path_list:
            _granularity = _parse_string(_p)[0]
            _date = _parse_string(_p)[1]
            u_ad_app_id_cnt = spark.read.format("delta").load(
                's3://' + BASE_BACKET + '/' + CREATIVE_PREFIX.format(_granularity) + '/date={}'.format(_date)).select(
                "ad_app_id").distinct().count()
            u_country_code_cnt = spark.read.format("delta").load(
                's3://' + BASE_BACKET + '/' + CREATIVE_PREFIX.format(_granularity) + '/date={}'.format(_date)).select(
                "country_code").distinct().count()
            u_ad_platform_id_cnt = spark.read.format("delta").load(
                's3://' + BASE_BACKET + '/' + CREATIVE_PREFIX.format(_granularity) + '/date={}'.format(_date)).select(
                "ad_platform_id").distinct().count()

            db_ad_app_id_cnt = \
                query(citus_dsn, BASE_CNT_SQL.format(dimension=AD_APP_ID, granularity=_granularity, date=_date))[0][0]
            db_country_code_cnt = \
                query(citus_dsn, BASE_CNT_SQL.format(dimension=COUNTRY_CODE, granularity=_granularity, date=_date))[0][
                    0]
            db_ad_platform_id_cnt = \
                query(citus_dsn, BASE_CNT_SQL.format(dimension=AD_PLATFORM_ID, granularity=_granularity, date=_date))[
                    0][0]
            try: 
                assert u_ad_app_id_cnt == db_ad_app_id_cnt, "u_ad_app_id_cnt: {} not equal db_ad_app_id_cnt: {} in {} - {}".format(
                    u_ad_app_id_cnt, db_ad_app_id_cnt, _granularity, _date)
                assert u_country_code_cnt == db_country_code_cnt, "u_country_code_cnt: {} not equal db_country_code_cnt: {} in {} - {}".format(
                    u_country_code_cnt, db_country_code_cnt, _granularity, _date)
                assert u_ad_platform_id_cnt == db_ad_platform_id_cnt, "u_ad_platform_id_cnt: {} not equal db_ad_platform_id_cnt: {} in {} - {}".format(
                    u_ad_platform_id_cnt, db_ad_platform_id_cnt, _granularity, _date)
            except AssertionError as msg:
                print msg


load_base_test()


In [0]:

BASE_BACKET = 'b2c-prod-data-pipeline-unified-market'
CREATIVE_PREFIX = "unified/market.creative-fact.v2/fact/granularity={}"
BASE_CNT_SQL = """SELECT COUNT(DISTINCT {dimension}) from  market.market_creative_adapp_fact_v1 where granularity='{granularity}' and date='{date}';"""
AD_APP_ID = 'ad_app_id'
COUNTRY_CODE = 'country_code'
AD_PLATFORM_ID = 'ad_platform_id'


def load_base_test(granularity, date):
    u_ad_app_id_cnt = spark.read.format("delta").load(
        's3://' + BASE_BACKET + '/' + CREATIVE_PREFIX.format(granularity) + '/date={}'.format(date)).select(
        "ad_app_id").distinct().count()
    u_country_code_cnt = spark.read.format("delta").load(
        's3://' + BASE_BACKET + '/' + CREATIVE_PREFIX.format(granularity) + '/date={}'.format(date)).select(
        "country_code").distinct().count()
    u_ad_platform_id_cnt = spark.read.format("delta").load(
        's3://' + BASE_BACKET + '/' + CREATIVE_PREFIX.format(granularity) + '/date={}'.format(date)).select(
        "ad_platform_id").distinct().count()

    db_ad_app_id_cnt = \
        query(citus_dsn, BASE_CNT_SQL.format(dimension=AD_APP_ID, granularity=granularity, date=date))[0][0]
    db_country_code_cnt = \
        query(citus_dsn, BASE_CNT_SQL.format(dimension=COUNTRY_CODE, granularity=granularity, date=date))[0][
            0]
    db_ad_platform_id_cnt = \
        query(citus_dsn, BASE_CNT_SQL.format(dimension=AD_PLATFORM_ID, granularity=granularity, date=date))[
            0][0]
    try:
        assert u_ad_app_id_cnt == db_ad_app_id_cnt, "u_ad_app_id_cnt: {} not equal db_ad_app_id_cnt: {} in {} - {}".format(
            u_ad_app_id_cnt, db_ad_app_id_cnt, granularity, date)
        assert u_country_code_cnt == db_country_code_cnt, "u_country_code_cnt: {} not equal db_country_code_cnt: {} in {} - {}".format(
            u_country_code_cnt, db_country_code_cnt, granularity, date)
        assert u_ad_platform_id_cnt == db_ad_platform_id_cnt, "u_ad_platform_id_cnt: {} not equal db_ad_platform_id_cnt: {} in {} - {}".format(
            u_ad_platform_id_cnt, db_ad_platform_id_cnt, granularity, date)
    except AssertionError as msg:
        print
        msg


load_base_test('monthly', '2020-01-31')

In [0]:
%%sh
PGPASSWORD='rn*Wh%osCl2C' psql -h internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com  -U citus_mkt_qa -d aa_mkt_db -p 7432 << EOF

 -- select creative_count from market.market_adapp_fact_v1 where granularity='monthly' AND device_code = 'ios-phone' AND  country_code = 'BR' AND date='2020-03-31' AND ad_app_id=284035177 and ad_platform_id=325;

select count(distinct country_code) from  market.market_creative_adapp_fact_v1 where granularity='weekly' and date='2015-04-11';

In [0]:

all_cnt = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-market/unified/market.creative-fact.v2/fact/granularity=monthly/date=2020-01-31/").count()
print all_cnt
ad_app_id_cnt  = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-market/unified/market.creative-fact.v2/fact/granularity=weekly/date=2015-04-11/").select("country_code").distinct().count()
print ad_app_id_cnt

country_code_cnt  = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-market/unified/market.creative-fact.v2/fact/granularity=monthly/date=2020-01-31/").select("country_code").distinct().count()
print country_code_cnt

ad_platform_id_cnt  = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-market/unified/market.creative-fact.v2/fact/granularity=monthly/date=2020-01-31/").select("ad_platform_id").distinct().count()
print ad_platform_id_cnt

In [0]:

# DB dependancy
from aadatapipelinecore.core.urn import Urn
import psycopg2
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
CITUS_MKT_HOSTS = [('internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com', 7432)]
CITUS_MKT_NAME = 'aa_mkt_db'
CITUS_MKT_ACCESS_ID = 'citus_mkt_qa'
CITUS_MKT_SECRET_KEY = 'rn*Wh%osCl2C'
CITUS_MKT_CONNECTION_CAPACITY = 10
CITUS_MKT_SEMAPHORE_TIMEOUT = 7201
citus_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=CITUS_MKT_NAME,
        user=CITUS_MKT_ACCESS_ID,
        host=CITUS_MKT_HOSTS[0][0],
        password=CITUS_MKT_SECRET_KEY,
        port=CITUS_MKT_HOSTS[0][1]
    )
)
def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result
sql = """select count(distinct ad_app_id) from  market.market_creative_adapp_fact_v1 where granularity='monthly' and date='2020-01-31';"""
print query(citus_dsn, sql)
###

In [0]:

all_cnt = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-market/unified/market.creative-fact.v2/fact/").groupBy("date","granularity").agg(F.countDistinct("ad_app_id").alias('ad_app_id_all')).cache()


In [0]:

all_app = all_cnt.filter("granularity in ('weekly','monthly')").select("ad_app_id_all").collect()
print sum([_[0] for _ in all_app])